In [1]:
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import LabelBinarizer
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.covariance import EllipticEnvelope
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.linear_model import SGDOneClassSVM
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.decomposition import PCA
from sklearn.kernel_approximation import Nystroem
from sklearn import metrics
from sklearn.svm import OneClassSVM
import time
import copy
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
import os,sys
import tqdm
import pickle
import time
import ast

In [2]:
dataPath = 'd:/mt_data/1212_withrw/encoded/t1/'
dictPath = 'd:/mt_data/1126_withrw/encoded/t1/'

In [3]:
devices = [ 'pi3', 'pi4_2G', 'pi4_4G']
tws = 60

In [4]:
features = [ 
    'system calls frequency_1gram', 
    'system calls tfidf_1gram',
    'system calls frequency_2gram', 
    'system calls tfidf_2gram',
    'system calls frequency_3gram',
    'system calls tfidf_3gram', 
    'system calls frequency_4gram', 
    'system calls tfidf_4gram',
    'system calls frequency_5gram',
    'system calls tfidf_5gram'
    ]

In [22]:
onepath = os.getcwd()+'/' +'onedata/'

In [24]:
tsv_name = onepath+'encoded_bow{}_{}_{}.csv'.format(d, tws, f)

In [27]:
tsv_df = pd.read_csv(tsv_name)

In [29]:
tsv_df['system calls tfidf_5gram']

0    [0.1549480754759887, 0.0, 0.0, 0.0, 0.0, 0.0, ...
Name: system calls tfidf_5gram, dtype: object

In [23]:
for d in devices:
    for f in features:
        tsv_name = dataPath+'encoded_bow{}_{}_{}.csv'.format(d, tws, f)
        tsv_df = pd.read_csv(tsv_name, sep='\t')
        tsv_df[0:1].to_csv(onepath+'encoded_bow{}_{}_{}.csv'.format(d, tws, f))

In [5]:
def read_feature_tsv(device, tw, ftname):
    tsv_name = dataPath+'encoded_bow{}_{}_{}.csv'.format(device, tw, ftname)
    tsv_df = pd.read_csv(tsv_name, sep='\t')
    feature = [ast.literal_eval(i) for i in tsv_df[ftname]]
    tsv_df[ftname] = feature
    normal = tsv_df[tsv_df.maltype=='normal'][ftname].tolist() 
    return normal, tsv_df

In [11]:
def read_feature_tsv(device, tw, ftname):
    tsv_name = dataPath+'encoded_bow{}_{}_{}.csv'.format(device, tw, ftname)
    tsv_df = pd.read_csv(tsv_name, sep='\t')
    feature = [ast.literal_eval(i) for i in tsv_df[ftname]]
    tsv_df[ftname] = feature
    normal = tsv_df[tsv_df.maltype=='normal'][ftname].tolist() 
    X_train, X_val = train_test_split(normal, test_size=.3, shuffle=False) 
    return X_train, X_val, tsv_df

In [19]:
def normal(device, tw, ftname):
    X_train, X_val, tsv_df = read_feature_tsv(device, tw, ftname)
    scaler = StandardScaler().fit(X_train)
    data = scaler.transform(tsv_df[ftname].tolist())
    normaled_df = pd.DataFrame([tsv_df['ids'].tolist(), tsv_df['maltype'].tolist(), data.tolist()]).transpose()
    scaledName = dataPath+'{}_{}_{}-scaled.pk'.format(device, tw, ftname)
    loc = open(scaledName,'wb')
    pickle.dump(scaler, loc)
    loc.close()
    ftname = ftname+'-scaled'
    normaled_df.columns = ['ids', 'maltype', ftname]
    dfName = dataPath+'encoded_bow{}_{}_{}.csv'.format(device, tw, ftname)
    normaled_df.to_csv(dfName, sep='\t', index=None)

In [9]:
def normal(device, tw, ftname):
    X_train, tsv_df = read_feature_tsv(device, tw, ftname)
    scaledName = dictPath+'{}_{}_{}-pcas.pk'.format(device, tw, ftname)
    loc = open(scaledName,'rb')
    scaler = pickle.load(loc)
    data = scaler.transform(X_train)
    normaled_df = pd.DataFrame([tsv_df['ids'].tolist(), tsv_df['maltype'].tolist(), data.tolist()]).transpose()
    ftname = ftname+'-pcas'
    normaled_df.columns = ['ids', 'maltype', ftname]
    dfName = dataPath+'encoded_bow{}_{}_{}.csv'.format(device, tw, ftname)
    normaled_df.to_csv(dfName, sep='\t', index=None)

In [11]:
features = [ 
    'system calls frequency_2gram', 
    'system calls tfidf_2gram',
    'system calls frequency_3gram',
    'system calls tfidf_3gram', 
    'system calls frequency_4gram', 
    'system calls tfidf_4gram',
    'system calls frequency_5gram',
    'system calls tfidf_5gram'
    ]

In [12]:
for device in devices:
    for ftname in features:
        normal(device, 60, ftname)